### Analysis of Everypol (after JSON to pd)

In [1]:
import os
import pandas as pd
import json
import ast
import openpyxl

In [2]:
def process_json_files_to_matrix(json_folder):
    """
    Process JSON files and create a matrix of filenames vs names present in files.
    
    Args:
        json_folder (str): Path to folder containing JSON files
        
    Returns:
        pd.DataFrame: Matrix with filenames as rows and all unique names as columns.
                     Values are boolean indicating if name is present in file.
    """
    
    all_names = set()
    file_list = [f for f in os.listdir(json_folder) if f.endswith(".json")]

    for filename in file_list:
        file_path = os.path.join(json_folder, filename)
        with open(file_path, "r") as file:
            try:
                data = json.load(file)
                
                if isinstance(data, dict):
                    data = [data]
                elif not isinstance(data, list):
                    data = []
                
                # Extract names
                all_names.update(entry["Name"] for entry in data if isinstance(entry, dict) and "Name" in entry)
            except (json.JSONDecodeError, TypeError):
                pass
    
    all_names = sorted(all_names)

    df = pd.DataFrame(columns=["Filename"] + all_names)

    for filename in file_list:
        file_path = os.path.join(json_folder, filename)
        with open(file_path, "r") as file:
            try:
                data = json.load(file)
                
                # Ensure data is a list
                if isinstance(data, dict):
                    data = [data]
                elif not isinstance(data, list):
                    data = []

                present_names = {entry["Name"] for entry in data if isinstance(entry, dict) and "Name" in entry}
            except (json.JSONDecodeError, TypeError):
                present_names = set()
        
        row = {"Filename": filename.replace(".json", "")}
        row.update({name: name in present_names for name in all_names})
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

    return df

### Assume MCAR

Where we don't have pol. emails etc., let's just assume that the data are missing at random. 

In [3]:
def clean_email_column_no_dedupe(df, column_name="email"):
    """
    Cleans the specified email column in a DataFrame by:
    1. Stripping whitespace, converting to lowercase, and removing commas.
    2. Dropping rows where the email contains only a single letter or symbol.
    3. Dropping rows where the email is NaN.
    4. Valid email

    Args:
        df (pd.DataFrame): The DataFrame to clean.
        column_name (str): The column to process (default: "email").

    Returns:
        pd.DataFrame: Cleaned DataFrame (modification done safely).
    """
    if column_name in df.columns:
        df = df.copy()
        df[column_name] = df[column_name].str.strip().str.lower().str.replace(",", "", regex=True).str.replace(" ", "")
        df = df[~df[column_name].str.match(r"^[A-Za-z,_-]$", na=False)]
        df = df.dropna(subset=[column_name])
        
        email_regex = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'

        df = df[df[column_name].str.match(email_regex, na=False)]

    return df

In [4]:
everypol_hibp = process_json_files_to_matrix("../data/everypol/everypol_hibp")
everypol_hibp.shape

(8535, 220)

In [5]:
everypol_hibp.head()

,Filename,000webhost,123RF,2844Breaches,500px,8fit,ABFRL,AKP,ActMobile,Acuity,...,Zacks,Zomato,Zynga,bigbasket,db8151dd,digiDirect,eThekwiniMunicipality,iMesh,ixigo,piZap
0,wouter.raskin@dekamer.be,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,k.verhoeven@tweedekamer.nl,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2,lcoffice@dphk.org,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,pk.sreemathi@sansad.nic.in,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,edurubio@parlamento.gub.uy,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [6]:
long_everypol_hibp = pd.melt(
    everypol_hibp,
    id_vars=['Filename'],
    var_name='Breach',
    value_name='Present'
)
long_everypol_hibp.shape

(1869165, 3)

In [7]:
long_everypol_hibp.to_csv("../data/everypol_hibp.csv", index=False)

In [8]:
breaches = pd.read_csv("../data/hipb_01_2025_breaches_data.csv")
breaches.head()

,Name,Title,Domain,BreachDate,AddedDate,ModifiedDate,PwnCount,Description,LogoPath,DataClasses,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree
0,000webhost,000webhost,000webhost.com,2015-03-01,2015-10-26T23:35:45Z,2017-12-10T21:44:27Z,14936670,"In approximately March 2015, the free web host...",https://haveibeenpwned.com/Content/Images/Pwne...,"['Email addresses', 'IP addresses', 'Names', '...",True,False,False,False,False,False,False
1,123RF,123RF,123rf.com,2020-03-22,2020-11-15T00:59:50Z,2020-11-15T01:07:10Z,8661578,"In March 2020, the stock photo site <a href=""h...",https://haveibeenpwned.com/Content/Images/Pwne...,"['Email addresses', 'IP addresses', 'Names', '...",True,False,False,False,False,False,False
2,126,126,126.com,2012-01-01,2016-10-08T07:46:05Z,2016-10-08T07:46:05Z,6414191,"In approximately 2012, it's alleged that the C...",https://haveibeenpwned.com/Content/Images/Pwne...,"['Email addresses', 'Passwords']",False,False,False,False,False,False,False
3,17Media,17,17app.co,2016-04-19,2016-07-08T01:55:03Z,2016-07-08T01:55:03Z,4009640,"In April 2016, customer data obtained from the...",https://haveibeenpwned.com/Content/Images/Pwne...,"['Device information', 'Email addresses', 'IP ...",True,False,False,False,False,False,False
4,17173,17173,17173.com,2011-12-28,2018-04-28T04:53:15Z,2018-04-28T04:53:15Z,7485802,"In late 2011, <a href=""https://news.softpedia....",https://haveibeenpwned.com/Content/Images/Pwne...,"['Email addresses', 'Passwords', 'Usernames']",False,False,False,False,False,False,False


In [9]:
breaches["DataClasses"] = breaches["DataClasses"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
all_categories = set(item for sublist in breaches["DataClasses"] for item in sublist)
expanded_df = pd.DataFrame(
    {category: breaches["DataClasses"].apply(lambda x: category in x) for category in all_categories}
)
breaches = pd.concat([breaches.drop(columns=["DataClasses"]), expanded_df], axis=1)

In [10]:
breaches_everypol_hibp = long_everypol_hibp.merge(breaches, left_on="Breach", right_on="Name", how="left")
breaches_everypol_hibp.head()

,Filename,Breach,Present,Name,Title,Domain,BreachDate,AddedDate,ModifiedDate,PwnCount,...,Loan information,Utility bills,Device usage tracking data,Mothers maiden names,Social media profiles,Biometric data,Credit cards,Drug habits,Purchases,Password hints
0,wouter.raskin@dekamer.be,000webhost,False,000webhost,000webhost,000webhost.com,2015-03-01,2015-10-26T23:35:45Z,2017-12-10T21:44:27Z,14936670,...,False,False,False,False,False,False,False,False,False,False
1,k.verhoeven@tweedekamer.nl,000webhost,False,000webhost,000webhost,000webhost.com,2015-03-01,2015-10-26T23:35:45Z,2017-12-10T21:44:27Z,14936670,...,False,False,False,False,False,False,False,False,False,False
2,lcoffice@dphk.org,000webhost,False,000webhost,000webhost,000webhost.com,2015-03-01,2015-10-26T23:35:45Z,2017-12-10T21:44:27Z,14936670,...,False,False,False,False,False,False,False,False,False,False
3,pk.sreemathi@sansad.nic.in,000webhost,False,000webhost,000webhost,000webhost.com,2015-03-01,2015-10-26T23:35:45Z,2017-12-10T21:44:27Z,14936670,...,False,False,False,False,False,False,False,False,False,False
4,edurubio@parlamento.gub.uy,000webhost,False,000webhost,000webhost,000webhost.com,2015-03-01,2015-10-26T23:35:45Z,2017-12-10T21:44:27Z,14936670,...,False,False,False,False,False,False,False,False,False,False


In [11]:
boolean_columns = breaches_everypol_hibp.select_dtypes(include=['bool'])
grouped_bool_counts = boolean_columns.groupby(breaches_everypol_hibp["Filename"]).sum().reset_index()
grouped_bool_counts.describe()

,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree,Driver's licenses,Vehicle details,Chat logs,...,Loan information,Utility bills,Device usage tracking data,Mothers maiden names,Social media profiles,Biometric data,Credit cards,Drug habits,Purchases,Password hints
count,8535.0,8535.0,8535.0,8535.0,8535.0,8535.0,8535.0,8535.0,8535.0,8535.0,...,8535.0,8535.0,8535.0,8535.0,8535.0,8535.0,8535.0,8535.0,8535.0,8535.0
mean,207.0,1.0,0.0,0.0,12.0,2.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,22.0,0.0,1.0,0.0,15.0,1.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,207.0,1.0,0.0,0.0,12.0,2.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,22.0,0.0,1.0,0.0,15.0,1.0
25%,207.0,1.0,0.0,0.0,12.0,2.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,22.0,0.0,1.0,0.0,15.0,1.0
50%,207.0,1.0,0.0,0.0,12.0,2.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,22.0,0.0,1.0,0.0,15.0,1.0
75%,207.0,1.0,0.0,0.0,12.0,2.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,22.0,0.0,1.0,0.0,15.0,1.0
max,207.0,1.0,0.0,0.0,12.0,2.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,22.0,0.0,1.0,0.0,15.0,1.0


In [12]:
grouped_bool_counts.head()

,Filename,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree,Driver's licenses,Vehicle details,...,Loan information,Utility bills,Device usage tracking data,Mothers maiden names,Social media profiles,Biometric data,Credit cards,Drug habits,Purchases,Password hints
0,(i)joice.george@sansad.nic.in,207,1,0,0,12,2,1,1,1,...,0,1,0,1,22,0,1,0,15,1
1,1.office@bjpanda.org,207,1,0,0,12,2,1,1,1,...,0,1,0,1,22,0,1,0,15,1
2,101bbb@naver.com,207,1,0,0,12,2,1,1,1,...,0,1,0,1,22,0,1,0,15,1
3,2016kimkj@gmail.com,207,1,0,0,12,2,1,1,1,...,0,1,0,1,22,0,1,0,15,1
4,2016kimss@gmail.com,207,1,0,0,12,2,1,1,1,...,0,1,0,1,22,0,1,0,15,1


In [13]:
pol_dat = pd.read_csv("../data/everypol/everypol_combined_legislature_data.csv", low_memory=False)
pol_dat.head()

,id,name,sort_name,email,twitter,facebook,group,group_id,area_id,area,...,n_unique_emails,person_count_legistype,ltype,url,cc,leg_start_date,lastmod,cc3,pop2024,lastmod_year
0,35ad9676-8485-4137-9a16-50f2844f3ab2,"Adhalrao Patil, Shri Shivaji","Adhalrao Patil, Shri Shivaji",shivajirao@sansad.nic.in,NaN,NaN,Shiv Sena,SS,shirur,Shirur,...,511,541,unicameral legislature,https://cdn.rawgit.com/everypolitician/everypo...,IN,2014-05-26,1557812040,IND,1.441720e+09,2019
1,f1f0f31d-ddb8-4681-82c0-4484d0d26ee3,"Adhikari, Shri Deepak (Dev)","Adhikari, Shri Deepak (Dev)",adhikari.deepak@sansad.nic.in,idevadhikari,IamTheDev,All India Trinamool Congress,AITC,ghatal,Ghatal,...,511,541,unicameral legislature,https://cdn.rawgit.com/everypolitician/everypo...,IN,2014-05-26,1557812040,IND,1.441720e+09,2019
2,bb68e3c6-de79-4d07-ad1f-90bb9a61ded0,"Adhikari, Shri Sisir Kumar","Adhikari, Shri Sisir Kumar",sisiradhikari76@yahoo.com,NaN,NaN,All India Trinamool Congress,AITC,kanthi,Kanthi,...,511,541,unicameral legislature,https://cdn.rawgit.com/everypolitician/everypo...,IN,2014-05-26,1557812040,IND,1.441720e+09,2019
3,c9a891c3-adb6-4581-b93b-72f9c00beacc,"Adhikari, Shri Suvendu","Adhikari, Shri Suvendu",adhikari.suvendu@sansad.nic.in,NaN,NaN,All India Trinamool Congress,AITC,tamluk,Tamluk,...,511,541,unicameral legislature,https://cdn.rawgit.com/everypolitician/everypo...,IN,2014-05-26,1557812040,IND,1.441720e+09,2019
4,5c22c70d-8317-4f94-97ec-2ccadd1acdf3,"Adityanath , Shri Yogi","Adityanath , Shri Yogi",yogi.adityanath@sansad.nic.in,NaN,NaN,Bharatiya Janata Party,BJP,gorakhpur,Gorakhpur,...,511,541,unicameral legislature,https://cdn.rawgit.com/everypolitician/everypo...,IN,2014-05-26,1557812040,IND,1.441720e+09,2019


In [14]:
pol_dat.shape

(25087, 35)

In [15]:
pol_dat = clean_email_column_no_dedupe(pol_dat)
pol_dat.shape

(13798, 35)

In [16]:
pol_hibp = pol_dat.merge(grouped_bool_counts, left_on="email", right_on="Filename", how="left")
pol_hibp.head()

,id,name,sort_name,email,twitter,facebook,group,group_id,area_id,area,...,Loan information,Utility bills,Device usage tracking data,Mothers maiden names,Social media profiles,Biometric data,Credit cards,Drug habits,Purchases,Password hints
0,35ad9676-8485-4137-9a16-50f2844f3ab2,"Adhalrao Patil, Shri Shivaji","Adhalrao Patil, Shri Shivaji",shivajirao@sansad.nic.in,NaN,NaN,Shiv Sena,SS,shirur,Shirur,...,0.0,1.0,0.0,1.0,22.0,0.0,1.0,0.0,15.0,1.0
1,f1f0f31d-ddb8-4681-82c0-4484d0d26ee3,"Adhikari, Shri Deepak (Dev)","Adhikari, Shri Deepak (Dev)",adhikari.deepak@sansad.nic.in,idevadhikari,IamTheDev,All India Trinamool Congress,AITC,ghatal,Ghatal,...,0.0,1.0,0.0,1.0,22.0,0.0,1.0,0.0,15.0,1.0
2,bb68e3c6-de79-4d07-ad1f-90bb9a61ded0,"Adhikari, Shri Sisir Kumar","Adhikari, Shri Sisir Kumar",sisiradhikari76@yahoo.com,NaN,NaN,All India Trinamool Congress,AITC,kanthi,Kanthi,...,0.0,1.0,0.0,1.0,22.0,0.0,1.0,0.0,15.0,1.0
3,c9a891c3-adb6-4581-b93b-72f9c00beacc,"Adhikari, Shri Suvendu","Adhikari, Shri Suvendu",adhikari.suvendu@sansad.nic.in,NaN,NaN,All India Trinamool Congress,AITC,tamluk,Tamluk,...,0.0,1.0,0.0,1.0,22.0,0.0,1.0,0.0,15.0,1.0
4,5c22c70d-8317-4f94-97ec-2ccadd1acdf3,"Adityanath , Shri Yogi","Adityanath , Shri Yogi",yogi.adityanath@sansad.nic.in,NaN,NaN,Bharatiya Janata Party,BJP,gorakhpur,Gorakhpur,...,0.0,1.0,0.0,1.0,22.0,0.0,1.0,0.0,15.0,1.0


In [17]:
pol_hibp.describe().round(2)

,row_id,leg_start_year,row_count,n_unique_emails,person_count_legistype,lastmod,pop2024,lastmod_year,IsVerified,IsFabricated,...,Loan information,Utility bills,Device usage tracking data,Mothers maiden names,Social media profiles,Biometric data,Credit cards,Drug habits,Purchases,Password hints
count,13798.00,13798.00,13798.00,13798.00,13798.00,1.379800e+04,1.326600e+04,13798.00,13128.0,13128.0,...,13128.0,13128.0,13128.0,13128.0,13128.0,13128.0,13128.0,13128.0,13128.0,13128.0
mean,263.89,2013.62,318.33,216.76,675.62,1.556704e+09,8.889324e+07,2018.99,207.0,1.0,...,0.0,1.0,0.0,1.0,22.0,0.0,1.0,0.0,15.0,1.0
std,174.66,3.72,196.91,170.04,511.54,1.800954e+06,2.722694e+08,0.07,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.00,1997.00,31.00,31.00,31.00,1.542919e+09,8.034100e+04,2018.00,207.0,1.0,...,0.0,1.0,0.0,1.0,22.0,0.0,1.0,0.0,15.0,1.0
25%,119.00,2012.00,158.00,78.00,267.00,1.556117e+09,6.378654e+06,2019.00,207.0,1.0,...,0.0,1.0,0.0,1.0,22.0,0.0,1.0,0.0,15.0,1.0
50%,215.00,2015.00,291.00,149.00,541.00,1.557305e+09,2.688913e+07,2019.00,207.0,1.0,...,0.0,1.0,0.0,1.0,22.0,0.0,1.0,0.0,15.0,1.0
75%,390.00,2016.00,432.00,314.00,964.00,1.557812e+09,6.855680e+07,2019.00,207.0,1.0,...,0.0,1.0,0.0,1.0,22.0,0.0,1.0,0.0,15.0,1.0
max,749.00,2019.00,686.00,646.00,1783.00,1.557904e+09,1.441720e+09,2019.00,207.0,1.0,...,0.0,1.0,0.0,1.0,22.0,0.0,1.0,0.0,15.0,1.0


In [18]:
columns_to_fill = list(all_categories) + ['IsVerified', 'IsFabricated', 'IsSensitive', 
                   'IsRetired', 'IsSpamList', 'IsMalware', 'IsSubscriptionFree']
pol_hibp[columns_to_fill] = pol_hibp[columns_to_fill].fillna(0)

In [19]:
pol_hibp.groupby(['country', 'legislature', 'leg_start_year']).mean(numeric_only=True).reset_index().describe().round(2)

,leg_start_year,row_id,row_count,n_unique_emails,person_count_legistype,lastmod,pop2024,lastmod_year,IsVerified,IsFabricated,...,Loan information,Utility bills,Device usage tracking data,Mothers maiden names,Social media profiles,Biometric data,Credit cards,Drug habits,Purchases,Password hints
count,114.00,114.00,114.00,114.00,114.00,1.140000e+02,1.050000e+02,114.00,114.00,114.00,...,114.0,114.00,114.0,114.00,114.00,114.0,114.00,114.0,114.00,114.00
mean,2012.75,348.47,214.21,116.27,478.28,1.556414e+09,3.887496e+07,2018.98,195.42,0.94,...,0.0,0.94,0.0,0.94,20.77,0.0,0.94,0.0,14.16,0.94
std,4.08,187.53,157.16,109.53,442.56,2.479202e+06,1.431282e+08,0.13,30.43,0.15,...,0.0,0.15,0.0,0.15,3.23,0.0,0.15,0.0,2.21,0.15
min,1997.00,1.00,31.00,31.00,31.00,1.542919e+09,8.034100e+04,2018.00,59.14,0.29,...,0.0,0.29,0.0,0.29,6.29,0.0,0.29,0.0,4.29,0.29
25%,2011.00,196.50,111.00,51.25,171.75,1.555979e+09,5.262079e+06,2019.00,202.41,0.98,...,0.0,0.98,0.0,0.98,21.51,0.0,0.98,0.0,14.67,0.98
50%,2014.00,365.50,170.00,73.00,292.00,1.557280e+09,1.031910e+07,2019.00,207.00,1.00,...,0.0,1.00,0.0,1.00,22.00,0.0,1.00,0.0,15.00,1.00
75%,2016.00,488.75,297.00,142.25,606.00,1.557718e+09,4.040918e+07,2019.00,207.00,1.00,...,0.0,1.00,0.0,1.00,22.00,0.0,1.00,0.0,15.00,1.00
max,2019.00,749.00,686.00,646.00,1783.00,1.557904e+09,1.441720e+09,2019.00,207.00,1.00,...,0.0,1.00,0.0,1.00,22.00,0.0,1.00,0.0,15.00,1.00


### Let's merge the rest (India, Eur.)

In [20]:
no_hibp = process_json_files_to_matrix("../data/no_hibp")
dk_hibp = process_json_files_to_matrix("../data/dk_hibp")
in_hibp = process_json_files_to_matrix("../data/india/ls_hibp")

bihar_hibp = process_json_files_to_matrix("../data/india/bihar_hibp")
tn_hibp = process_json_files_to_matrix("../data/india/tn_hibp")
up_hibp = process_json_files_to_matrix("../data/india/up_hibp")
hp_hibp = process_json_files_to_matrix("../data/india/hp_hibp")
del_hibp = process_json_files_to_matrix("../data/india/delhi_hibp")

In [21]:
# Make it long
long_no_hibp    = pd.melt(no_hibp, id_vars=['Filename'], var_name='Breach', value_name='Present')
long_dk_hibp    = pd.melt(dk_hibp, id_vars=['Filename'], var_name='Breach', value_name='Present')
long_in_hibp    = pd.melt(in_hibp, id_vars=['Filename'], var_name='Breach', value_name='Present')

long_bihar_hibp = pd.melt(bihar_hibp, id_vars=['Filename'], var_name='Breach', value_name='Present')
long_tn_hibp    = pd.melt(tn_hibp, id_vars=['Filename'], var_name='Breach', value_name='Present')
long_up_hibp    = pd.melt(up_hibp, id_vars=['Filename'], var_name='Breach', value_name='Present')
long_hp_hibp    = pd.melt(hp_hibp, id_vars=['Filename'], var_name='Breach', value_name='Present')
long_del_hibp   = pd.melt(del_hibp, id_vars=['Filename'], var_name='Breach', value_name='Present')

In [22]:
# Join to breaches
breaches_no = long_no_hibp.merge(breaches, left_on="Breach", right_on="Name", how="left")
breaches_dk = long_dk_hibp.merge(breaches, left_on="Breach", right_on="Name", how="left")
breaches_in = long_in_hibp.merge(breaches, left_on="Breach", right_on="Name", how="left")

breaches_bihar = long_bihar_hibp.merge(breaches, left_on="Breach", right_on="Name", how="left")
breaches_tn    = long_tn_hibp.merge(breaches, left_on="Breach", right_on="Name", how="left")
breaches_up    = long_up_hibp.merge(breaches, left_on="Breach", right_on="Name", how="left")
breaches_hp    = long_hp_hibp.merge(breaches, left_on="Breach", right_on="Name", how="left")
breaches_del   = long_del_hibp.merge(breaches, left_on="Breach", right_on="Name", how="left")

In [23]:
breaches_no.head()

,Filename,Breach,Present,Name,Title,Domain,BreachDate,AddedDate,ModifiedDate,PwnCount,...,Loan information,Utility bills,Device usage tracking data,Mothers maiden names,Social media profiles,Biometric data,Credit cards,Drug habits,Purchases,Password hints
0,sandra.bruflot@stortinget.no,Adapt,False,Adapt,Adapt,adapt.io,2018-11-05,2018-11-22T19:43:06Z,2018-11-22T19:43:06Z,9363740,...,False,False,False,False,True,False,False,False,False,False
1,torbjorn.vereide@stortinget.no,Adapt,False,Adapt,Adapt,adapt.io,2018-11-05,2018-11-22T19:43:06Z,2018-11-22T19:43:06Z,9363740,...,False,False,False,False,True,False,False,False,False,False
2,marius.arion.nilsen@stortinget.no,Adapt,False,Adapt,Adapt,adapt.io,2018-11-05,2018-11-22T19:43:06Z,2018-11-22T19:43:06Z,9363740,...,False,False,False,False,True,False,False,False,False,False
3,torstein.solberg@stortinget.no,Adapt,False,Adapt,Adapt,adapt.io,2018-11-05,2018-11-22T19:43:06Z,2018-11-22T19:43:06Z,9363740,...,False,False,False,False,True,False,False,False,False,False
4,aslaug.sem-jacobsen@stortinget.no,Adapt,False,Adapt,Adapt,adapt.io,2018-11-05,2018-11-22T19:43:06Z,2018-11-22T19:43:06Z,9363740,...,False,False,False,False,True,False,False,False,False,False


In [24]:
# Group by email and sum the bool cols
no_breach_count = breaches_no.select_dtypes(include=['bool']).groupby(breaches_no["Filename"]).sum().reset_index()
dk_breach_count = breaches_dk.select_dtypes(include=['bool']).groupby(breaches_dk["Filename"]).sum().reset_index()
in_breach_count = breaches_in.select_dtypes(include=['bool']).groupby(breaches_in["Filename"]).sum().reset_index()

bihar_breach_count = breaches_bihar.select_dtypes(include=['bool']).groupby(breaches_bihar["Filename"]).sum().reset_index()
tn_breach_count = breaches_tn.select_dtypes(include=['bool']).groupby(breaches_tn["Filename"]).sum().reset_index()
up_breach_count = breaches_up.select_dtypes(include=['bool']).groupby(breaches_up["Filename"]).sum().reset_index()
hp_breach_count = breaches_hp.select_dtypes(include=['bool']).groupby(breaches_hp["Filename"]).sum().reset_index()
del_breach_count = breaches_del.select_dtypes(include=['bool']).groupby(breaches_del["Filename"]).sum().reset_index()

In [25]:
# Norway
no_parl = pd.read_csv("../data/no/no_parliament.csv")
no_parl = clean_email_column_no_dedupe(no_parl)
print(no_parl.shape)

# Denmark
wb = openpyxl.load_workbook("../data/danish_parliament_1_2025.xlsx")
ws = wb.active  
data = list(ws.values)
dk_parl = pd.DataFrame(data)
dk_parl.columns = dk_parl.iloc[0]
dk_parl = dk_parl[1:].reset_index(drop=True)
dk_parl = clean_email_column_no_dedupe(dk_parl)
print(dk_parl.shape)

# Bihar
with open('../data/india/bihar/bihar.txt', 'r', encoding='utf-8') as file:
    lines = file.read().splitlines()
    lines = [line.split('\t') for line in lines]

bihar_df = pd.DataFrame(lines, columns=['sr_no', 'photo', 'constituency', 'name', 'gender', 'party', 'contact', 'email'])
bihar_df = clean_email_column_no_dedupe(bihar_df)
print(bihar_df.shape)

# UP
up_df = pd.read_csv("../data/india/up/up_18_mlas.csv", usecols=['email'], encoding="utf-8", quotechar='"', sep=",", engine="python")
up_df = clean_email_column_no_dedupe(up_df)
print(up_df.shape)

# HP
hp_df = pd.read_csv("../data/india/hp_14.csv")
hp_df.columns = hp_df.columns.str.lower()
hp_df = clean_email_column_no_dedupe(hp_df)
print(hp_df.shape)

# TN
tn_df = pd.read_csv("../data/india/tn/tn.csv")
tn_df.rename(columns={"Email Address": "email"}, inplace=True)
tn_df = clean_email_column_no_dedupe(tn_df)
print(tn_df.shape)

# Delhi
del_df = pd.read_csv("../data/india/delhi/delhi_7th_assembly.csv")
del_df.rename(columns={"Email": "email"}, inplace=True)
del_df = clean_email_column_no_dedupe(del_df)
print(del_df.shape)

# India
in_df = pd.read_csv("../data/india/ls_long.csv")
in_df = clean_email_column_no_dedupe(in_df)
print(in_df.shape)

(174, 3)
(186, 10)
(241, 8)
(106, 1)
(68, 7)
(233, 6)
(43, 8)
(4403, 43)


/Users/soodoku/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [26]:
# Add metadata
no_parl = no_parl.assign(country='Norway', year=2025, legislature='National Legislature')
dk_parl = dk_parl.assign(country='Denmark', year=2025, legislature='National Legislature')
in_df = in_df.assign(country='India', year=2025, legislature='National Legislature')

bihar_df = bihar_df.assign(country='India', year=2025, legislature='Bihar Legislature')
tn_df = tn_df.assign(country='India', year=2025, legislature='Tamil Nadu State Legislature')
up_df = up_df.assign(country='India', year=2025, legislature='UP State Legislature')
hp_df = hp_df.assign(country='India', year=2025, legislature='HP Legislature')
del_df = del_df.assign(country='India', year=2025, legislature='Delhi Legislature')

### Join

In [27]:
no_all = no_parl.merge(no_breach_count, how = "left", left_on = "email", right_on = "Filename").fillna(0)
dk_all = dk_parl.merge(dk_breach_count, how = "left", left_on = "Email", right_on = "Filename").fillna(0)
in_all = in_df.merge(in_breach_count, how = "left", left_on = "email", right_on = "Filename").fillna(0)

bihar_all = bihar_df.merge(bihar_breach_count, how = "left", left_on = "email", right_on = "Filename").fillna(0)
tn_all = tn_df.merge(tn_breach_count, how = "left", left_on = "email", right_on = "Filename").fillna(0)
up_all = up_df.merge(up_breach_count, how = "left", left_on = "email", right_on = "Filename").fillna(0)
hp_all = hp_df.merge(hp_breach_count, how = "left", left_on = "email", right_on = "Filename").fillna(0)
del_all = del_df.merge(del_breach_count, how = "left", left_on = "email", right_on = "Filename").fillna(0)

### India is in long. Get it back to MP level

In [28]:
in_grouped = (
    in_all.groupby(['mpsno', 'source_file'], as_index=False)
    .sum(numeric_only=True)
)
in_grouped.head()

,mpsno,source_file,lastLoksabha,age,noOfTerms,numberOfSons,numberOfDaughters,currentPageNumber,perPageSize,totalElements,...,Loan information,Utility bills,Device usage tracking data,Mothers maiden names,Social media profiles,Biometric data,Credit cards,Drug habits,Purchases,Password hints
0,1,ls_13,14,77.0,2.0,2.0,1.0,1,551,551,...,2.0,0.0,0.0,0.0,37.0,0.0,2.0,0.0,38.0,2.0
1,1,ls_14,14,77.0,2.0,2.0,1.0,1,554,554,...,2.0,0.0,0.0,0.0,37.0,0.0,2.0,0.0,38.0,2.0
2,2,ls_12,14,53.0,3.0,2.0,0.0,1,529,529,...,2.0,0.0,0.0,0.0,37.0,0.0,2.0,0.0,38.0,2.0
3,2,ls_13,14,53.0,3.0,2.0,0.0,1,551,551,...,2.0,0.0,0.0,0.0,37.0,0.0,2.0,0.0,38.0,2.0
4,2,ls_14,14,53.0,3.0,2.0,0.0,1,554,554,...,2.0,0.0,0.0,0.0,37.0,0.0,2.0,0.0,38.0,2.0


In [29]:
in_grouped.shape

(2684, 165)

In [30]:
no_all.describe().round(2)

,year,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree,Driver's licenses,Vehicle details,...,Loan information,Utility bills,Device usage tracking data,Mothers maiden names,Social media profiles,Biometric data,Credit cards,Drug habits,Purchases,Password hints
count,174.0,174.0,174.0,174.0,174.0,174.0,174.0,174.0,174.0,174.0,...,174.0,174.0,174.0,174.0,174.0,174.0,174.0,174.0,174.0,174.0
mean,2025.0,26.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,1.0,1.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,2025.0,26.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,1.0,1.0
25%,2025.0,26.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,1.0,1.0
50%,2025.0,26.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,1.0,1.0
75%,2025.0,26.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,1.0,1.0
max,2025.0,26.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,1.0,1.0


In [31]:
dk_all.describe().round(2)

,year,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree,Driver's licenses,Vehicle details,...,Loan information,Utility bills,Device usage tracking data,Mothers maiden names,Social media profiles,Biometric data,Credit cards,Drug habits,Purchases,Password hints
count,186.0,186.00,186.0,186.0,186.0,186.00,186.00,186.0,186.0,186.0,...,186.0,186.0,186.0,186.0,186.00,186.0,186.0,186.0,186.0,186.00
mean,2025.0,36.77,0.0,0.0,0.0,3.87,0.97,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.71,0.0,0.0,0.0,0.0,0.97
std,0.0,6.73,0.0,0.0,0.0,0.71,0.18,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.59,0.0,0.0,0.0,0.0,0.18
min,2025.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
25%,2025.0,38.00,0.0,0.0,0.0,4.00,1.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,9.00,0.0,0.0,0.0,0.0,1.00
50%,2025.0,38.00,0.0,0.0,0.0,4.00,1.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,9.00,0.0,0.0,0.0,0.0,1.00
75%,2025.0,38.00,0.0,0.0,0.0,4.00,1.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,9.00,0.0,0.0,0.0,0.0,1.00
max,2025.0,38.00,0.0,0.0,0.0,4.00,1.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,9.00,0.0,0.0,0.0,0.0,1.00


In [32]:
in_grouped.groupby(['source_file']).mean()

,mpsno,lastLoksabha,age,noOfTerms,numberOfSons,numberOfDaughters,currentPageNumber,perPageSize,totalElements,totalPages,...,Loan information,Utility bills,Device usage tracking data,Mothers maiden names,Social media profiles,Biometric data,Credit cards,Drug habits,Purchases,Password hints
source_file,,,,,,,,,,,,,,,,,,,,,
ls_12,793.064356,17.693069,90.173267,5.435644,1.702970,1.599010,1.198020,633.752475,633.752475,1.198020,...,2.376238,0.0,0.0,0.0,43.960396,0.0,2.376238,0.0,45.148515,2.376238
ls_13,310.280112,16.943978,86.899160,4.221289,1.627451,1.526611,1.168067,643.605042,643.605042,1.168067,...,2.324930,0.0,0.0,0.0,43.011204,0.0,2.324930,0.0,44.173669,2.324930
ls_14,1960.614286,19.660714,89.392857,4.757143,1.664286,1.525000,1.282143,710.307143,710.307143,1.282143,...,2.564286,0.0,0.0,0.0,47.439286,0.0,2.564286,0.0,48.721429,2.564286
ls_15,3373.632768,22.887006,95.223164,4.220339,1.864407,1.740113,1.435028,733.299435,733.299435,1.435028,...,2.870056,0.0,0.0,0.0,53.096045,0.0,2.870056,0.0,54.531073,2.870056
ls_16,4048.672377,28.788009,108.526767,4.115632,2.027837,1.873662,1.745182,893.533191,893.533191,1.745182,...,3.490364,0.0,0.0,0.0,64.571734,0.0,3.490364,0.0,66.316916,3.490364
ls_17,4500.073022,33.103448,113.336714,3.819473,2.206897,1.957404,1.947262,1049.574037,1049.574037,1.947262,...,3.894523,0.0,0.0,0.0,72.048682,0.0,3.894523,0.0,73.995943,3.894523
ls_18,4998.207156,37.355932,118.583804,4.256121,1.664783,1.529190,2.075330,1128.979284,1128.979284,2.075330,...,4.150659,0.0,0.0,0.0,76.787194,0.0,4.150659,0.0,78.862524,4.150659


In [33]:
bihar_all.describe().round(2)

,year,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree,Driver's licenses,Vehicle details,...,Loan information,Utility bills,Device usage tracking data,Mothers maiden names,Social media profiles,Biometric data,Credit cards,Drug habits,Purchases,Password hints
count,241.0,241.0,241.0,241.0,241.0,241.0,241.0,241.0,241.0,241.0,...,241.0,241.0,241.0,241.0,241.0,241.0,241.0,241.0,241.0,241.0
mean,2025.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,2025.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,2025.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,2025.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,2025.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,2025.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
tn_all.describe().round(2)

,Photo,year,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree,Driver's licenses,...,Loan information,Utility bills,Device usage tracking data,Mothers maiden names,Social media profiles,Biometric data,Credit cards,Drug habits,Purchases,Password hints
count,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0,...,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0,233.0
mean,0.0,2025.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,2025.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
25%,0.0,2025.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
50%,0.0,2025.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
75%,0.0,2025.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
max,0.0,2025.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [35]:
hp_all.describe().round(2)

,photo,mobile,tele.no./res.no..,year,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,...,Loan information,Utility bills,Device usage tracking data,Mothers maiden names,Social media profiles,Biometric data,Credit cards,Drug habits,Purchases,Password hints
count,68.0,6.800000e+01,68.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0,...,68.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0,68.0
mean,0.0,9.416969e+09,0.0,2025.0,37.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,4.0,0.0
std,0.0,5.106352e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,7.018000e+09,0.0,2025.0,37.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,4.0,0.0
25%,0.0,9.418022e+09,0.0,2025.0,37.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,4.0,0.0
50%,0.0,9.418154e+09,0.0,2025.0,37.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,4.0,0.0
75%,0.0,9.805725e+09,0.0,2025.0,37.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,4.0,0.0
max,0.0,9.882812e+09,0.0,2025.0,37.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,4.0,0.0


In [36]:
up_all.describe().round(2)

,year,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree,Driver's licenses,Vehicle details,...,Loan information,Utility bills,Device usage tracking data,Mothers maiden names,Social media profiles,Biometric data,Credit cards,Drug habits,Purchases,Password hints
count,106.0,106.0,106.0,106.0,106.0,106.0,106.0,106.0,106.0,106.0,...,106.0,106.0,106.0,106.0,106.0,106.0,106.0,106.0,106.0,106.0
mean,2025.0,30.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,4.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,2025.0,30.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,4.0,0.0
25%,2025.0,30.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,4.0,0.0
50%,2025.0,30.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,4.0,0.0
75%,2025.0,30.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,4.0,0.0
max,2025.0,30.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,4.0,0.0


In [37]:
del_all.describe().round(2)

,AC_NO,year,IsVerified,IsFabricated,IsSensitive,IsRetired,IsSpamList,IsMalware,IsSubscriptionFree,Driver's licenses,...,Loan information,Utility bills,Device usage tracking data,Mothers maiden names,Social media profiles,Biometric data,Credit cards,Drug habits,Purchases,Password hints
count,43.00,43.0,43.00,43.00,43.0,43.0,43.00,43.00,43.0,43.0,...,43.0,43.0,43.0,43.0,43.00,43.0,43.0,43.0,43.00,43.00
mean,44.47,2025.0,32.23,0.98,0.0,0.0,2.93,0.98,0.0,0.0,...,0.0,0.0,0.0,0.0,5.86,0.0,0.0,0.0,3.91,0.98
std,16.26,0.0,5.03,0.15,0.0,0.0,0.46,0.15,0.0,0.0,...,0.0,0.0,0.0,0.0,0.91,0.0,0.0,0.0,0.61,0.15
min,18.00,2025.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00
25%,29.50,2025.0,33.00,1.00,0.0,0.0,3.00,1.00,0.0,0.0,...,0.0,0.0,0.0,0.0,6.00,0.0,0.0,0.0,4.00,1.00
50%,45.00,2025.0,33.00,1.00,0.0,0.0,3.00,1.00,0.0,0.0,...,0.0,0.0,0.0,0.0,6.00,0.0,0.0,0.0,4.00,1.00
75%,58.50,2025.0,33.00,1.00,0.0,0.0,3.00,1.00,0.0,0.0,...,0.0,0.0,0.0,0.0,6.00,0.0,0.0,0.0,4.00,1.00
max,70.00,2025.0,33.00,1.00,0.0,0.0,3.00,1.00,0.0,0.0,...,0.0,0.0,0.0,0.0,6.00,0.0,0.0,0.0,4.00,1.00
